In [1]:
ENV["JULIA_PKG_USE_CLI_GIT"] = "true"
cd("/leonardo_work/Sis25_degironc_0/apol/aceconverge2025/")
pwd()

import Pkg
Pkg.activate(".")
using ACEpotentials, LinearAlgebra

  Activating project at `/leonardo_work/Sis25_degironc_0/apol/aceconverge2025`


# Basis checking

## Carbon

## Silicon 

In [2]:
basis_bin = Dict()
orders = [2,3,4]
degrees = [[24,20],[24,20,16],[24,20,16,12]]
basis_tags = ["24.20","24.20.16","24.20.16.12"]

for (i,ord) in enumerate(orders)
    basis = ACE1x.ace_basis(
        elements = [:Si],
        order = ord,
        totaldegree = degrees[i],
        rcut = 6.0,
        r0 = 2.20707071, # for silicon dataset
        pure = false)
    basis_bin[basis_tags[i]] = basis
    println("basis_bin[\"" * basis_tags[i] * "\"]")
    println(length(basis))
end

# basis_bin_pure = Dict()
# for (i,ord) in enumerate(orders)
#     basis = ACE1x.ace_basis(
#         elements = [:Si],
#         order = ord,
#         totaldegree = degrees[i],
#         rcut = 6.0,
#         r0 = 2.20707071, # for silicon dataset
#         pure = true)
#     basis_bin_pure[basis_tags[i]] = basis
#     println("basis_bin[\"" * basis_tags[i] * "\"]")
#     println(length(basis))
# end

basis_bin["24.20"]
297
basis_bin["24.20.16"]
802
basis_bin["24.20.16.12"]
929


# Error extraction from complete potentials

## Out-of-domain

In [55]:
vset = "Si2"
dsets = ["Si2", "Si3", "Si4", "Si5"]
ecost = "50.0"
elossweight = parse(Float64, ecost)
weights = Dict("default" => Dict("E" => elossweight, "F" => 1.0))
basisvals = ["24.20","24.20.16","24.20.16.12"]
Vref = OneBody(:Si => -7881.32677981122) # Si2 isolated energy
datakeys = (energy_key = "energy", force_key = "forces")
val_data_dir = "./datasets/$(vset).xyz"; val_data = read_extxyz(val_data_dir)
val_atoms = [ACEpotentials.AtomsData(v; weights=weights, v_ref=Vref, datakeys...) for v in val_data]

pots = Dict()
pots_pure = Dict()
errs = Dict("train" => Dict(), "val" => Dict())
errs_pure = Dict("train" => Dict(), "val" => Dict())

for dset in dsets
    train_data_dir = "./datasets/$(dset).xyz"; train_data = read_extxyz(train_data_dir)
    train_atoms = [ACEpotentials.AtomsData(t; weights=weights, v_ref=Vref, datakeys...) for t in train_data]
    for basisval in basisvals
        # Load potential
        pots["$(dset)_$(basisval)"] = load_potential("./acejulia/$(dset)/$(basisval)/ecost$(ecost)/potential.json")
        pots_pure["$(dset)_$(basisval)"] = load_potential("./acejulia/$(dset)_purify/$(basisval)/ecost$(ecost)/potential.json")
        # Compute errors
        errs["train"]["$(dset)_$(basisval)"] = ACEpotentials.linear_errors(train_atoms, pots["$(dset)_$(basisval)"])
        errs_pure["train"]["$(dset)_$(basisval)"] = ACEpotentials.linear_errors(train_atoms, pots_pure["$(dset)_$(basisval)"])
        errs["val"]["$(dset)_$(basisval)"] = ACEpotentials.linear_errors(val_atoms, pots["$(dset)_$(basisval)"])
        errs_pure["val"]["$(dset)_$(basisval)"] = ACEpotentials.linear_errors(val_atoms, pots_pure["$(dset)_$(basisval)"])
    end
end


This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.


This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.

┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.523 │    0.051 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.523 │    0.051 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.330 │    0.015 │   0.000 │
├─────────┼─────────┼──────────

[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table



JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.


This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.

┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.523 │    0.051 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.523 │    0.051 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.330 │    0.015 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.330 │    0.015 │   0.00

[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table



JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.


This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.

┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.523 │    0.051 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.523 │    0.051 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.330 │    0.015 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.330 │    0.015 │   0.00

...

[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table



This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.

┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  24.424 │    0.463 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  24.424 │    0.463 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  19.875 │    0.338 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  19.875 │    0.338 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘


[ Info: RMSE Table
[ Info: MAE Table


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  20.280 │    0.506 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  20.280 │    0.506 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  16.300 │    0.368 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  16.300 │    0.368 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬──────────┬──────────┬─────────┐
│    Type │  E [meV] │ F [eV/A] │ V [meV] │
├─────────┼──────────┼──────────┼─────────┤
│ default │ 1396.295 │    4.063 │   0.000 │
├─────────┼──────────┼──────────┼─────────┤
│     set │ 1396.295 │    4.063 │   0.000 │
└─────────┴──────────┴──────────┴─────────┘
┌─────────┬──────────┬──────────┬─────────┐
│    Type │  E [meV] │ F [eV/A] │ V [meV] │
├─

[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table


In [56]:
entries = length(keys(errs["val"]))
en_err = zeros(entries)
f_err = zeros(entries)
en_err_pure = zeros(entries)
f_err_pure = zeros(entries)

for (i,tsetbset) in enumerate(sort(collect(keys(errs["val"]))))
    # println(tsetbset) # uncomment to understand the structure 
    en_err[i] = errs["val"][tsetbset]["mae"]["default"]["E"]
    f_err[i] = errs["val"][tsetbset]["mae"]["default"]["F"]
    en_err_pure[i] = errs_pure["val"][tsetbset]["mae"]["default"]["E"]
    f_err_pure[i] = errs_pure["val"][tsetbset]["mae"]["default"]["F"]
end

en_err = reshape(en_err, (3, 4))' # transposed with (')
f_err = reshape(f_err, (3, 4))'
en_err_pure = reshape(en_err_pure, (3, 4))'
f_err_pure = reshape(f_err_pure, (3, 4))'

4×3 adjoint(::Matrix{Float64}) with eltype Float64:
 0.0146319  0.0146319  0.0146319
 1.65766    1.65766    1.65766
 1.01954    1.06637    1.06609
 1.00143    1.05569    1.05584

In [57]:
open("./acejulia/si_errors/ood_vset$(vset)_en_err.dat", "w") do io
    for row in eachrow(en_err)
        for val in row
            @printf(io, "%.16f ", val)
        end
        println(io)
    end
end
open("./acejulia/si_errors/ood_vset$(vset)_f_err.dat", "w") do io
    for row in eachrow(f_err)
        for val in row
            @printf(io, "%.16f ", val)
        end
        println(io)
    end
end
open("./acejulia/si_errors/ood_vset$(vset)_en_err_pure.dat", "w") do io
    for row in eachrow(en_err_pure)
        for val in row
            @printf(io, "%.16f ", val)
        end
        println(io)
    end
end
open("./acejulia/si_errors/ood_vset$(vset)_f_err_pure.dat", "w") do io
    for row in eachrow(f_err_pure)
        for val in row
            @printf(io, "%.16f ", val)
        end
        println(io)
    end
end

In [42]:
# using Printf

# for row in eachrow(en_err)
#     for val in row
#         @printf("%.16f ", val)
#     end
#     println()
# end

In [15]:
errs["val"]["Si4_24.20"]["mae"]["default"]["E"]
errs_pure["val"]["Si4_24.20"]["mae"]["default"]["E"]

1.8509284013241813

In [59]:
## Data load ## 
dset = "Si5"
ecost = "50.0"
basisvals = ["24.20","24.20.16","24.20.16.12"]
pot = load_potential("./acejulia/$(dset)/$(basisvals[3])/ecost$(ecost)/potential.json");
pot_pure = load_potential("./acejulia/$(dset)_purify/$(basisvals[3])/ecost$(ecost)/potential.json");

# Validation data
Vref = OneBody(:Si => -7881.32677981122) # Si2 isolated energy
elossweight = parse(Float64, ecost)
datakeys = (energy_key = "energy", force_key = "force")
weights = Dict("default" => Dict("E" => elossweight, "F" => 1.0))
train_data_dir = "./datasets/$(dset).xyz"; train_data = read_extxyz(train_data_dir)
test_data_dir = "./datasets/Si5.xyz"; test_data = read_extxyz(test_data_dir)
train = [ACEpotentials.AtomsData(t; weights=weights, v_ref=Vref, datakeys...) for t in train_data]
test = [ACEpotentials.AtomsData(v; weights=weights, v_ref=Vref, datakeys...) for v in test_data]

## Error extraction ## 
trainerrs = ACEpotentials.linear_errors(train, pot)
trainerrs_pure = ACEpotentials.linear_errors(train, pot_pure)
testerrs = ACEpotentials.linear_errors(test, pot)
testerrs_pure = ACEpotentials.linear_errors(test, pot_pure)


This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.


This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.

┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  24.424 │    0.000 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  24.424 │    0.000 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  19.875 │    0.000 │   0.000 │
├─────────┼─────────┼──────────

[ Info: RMSE Table
[ Info: MAE Table


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  20.280 │    0.000 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  20.280 │    0.000 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  16.300 │    0.000 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  16.300 │    0.000 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  24.424 │    0.000 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  24.424 │    0.000 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼

[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  20.280 │    0.000 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  20.280 │    0.000 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  16.300 │    0.000 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  16.300 │    0.000 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘


[ Info: RMSE Table
[ Info: MAE Table


Dict{String, Dict{String, Any}} with 2 entries:
  "rmse" => Dict("default"=>Dict("V"=>0.0, "E"=>0.0202798, "F"=>0.0), "set"=>Di…
  "mae"  => Dict("default"=>Dict("V"=>0.0, "E"=>0.0163002, "F"=>0.0), "set"=>Di…

## In-domain

In [2]:
function export_dimers_to_dat(dimers_dict; filename)
    for ((z1, z0), (rr, v01)) in dimers_dict
        open(filename, "w") do io
            for (r, v) in zip(rr, v01)
                println(io, "$r $v")
            end
        end
    end
end

export_dimers_to_dat (generic function with 1 method)

In [ ]:
vset = "Si2"
dsets = ["Si23", "Si234", "Si2345"]
ecost = "50.0"
elossweight = parse(Float64, ecost)
weights = Dict("default" => Dict("E" => elossweight, "F" => 1.0))
basisvals = ["24.20","24.20.16","24.20.16.12"]
Vref = OneBody(:Si => -7881.32677981122) # Si2 isolated energy
datakeys = (energy_key = "energy", force_key = "forces")
val_data_dir = "./datasets/$(vset).xyz"; val_data = read_extxyz(val_data_dir)
val_atoms = [ACEpotentials.AtomsData(v; weights=weights, v_ref=Vref, datakeys...) for v in val_data]

pots = Dict()
pots_pure = Dict()
errs = Dict("train" => Dict(), "val" => Dict())
errs_pure = Dict("train" => Dict(), "val" => Dict())

for dset in dsets
    train_data_dir = "./datasets/$(dset).xyz"; train_data = read_extxyz(train_data_dir)
    train_atoms = [ACEpotentials.AtomsData(t; weights=weights, v_ref=Vref, datakeys...) for t in train_data]
    for basisval in basisvals
        pots["$(dset)_$(basisval)"] = load_potential("./acejulia/$(dset)/$(basisval)/ecost$(ecost)/potential.json")
        pots_pure["$(dset)_$(basisval)"] = load_potential("./acejulia/$(dset)_purify/$(basisval)/ecost$(ecost)/potential.json")

        # Errors
        errs["train"]["$(dset)_$(basisval)"] = ACEpotentials.linear_errors(train_atoms, pots["$(dset)_$(basisval)"])
        errs_pure["train"]["$(dset)_$(basisval)"] = ACEpotentials.linear_errors(train_atoms, pots_pure["$(dset)_$(basisval)"])
        errs["val"]["$(dset)_$(basisval)"] = ACEpotentials.linear_errors(val_atoms, pots["$(dset)_$(basisval)"])
        errs_pure["val"]["$(dset)_$(basisval)"] = ACEpotentials.linear_errors(val_atoms, pots_pure["$(dset)_$(basisval)"])

        # Dimers
        # DSI = ACEpotentials.dimers(pots["$(dset)_$(basisval)"], [:Si,]; rr = range(0.529177, 7.0, length=200))
        # D = ACEpotentials.dimers(pots_pure["$(dset)_$(basisval)"], [:Si,]; rr = range(0.529177, 7.0, length=200))
        # export_dimers_to_dat(DSI, filename="./acejulia/$(dset)/$(basisval)/ecost$(ecost)/dimers.dat")
        # export_dimers_to_dat(D, filename="./acejulia/$(dset)_purify/$(basisval)/ecost$(ecost)/dimercurve.dat")
    end
end


This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.


This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.

┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.605 │    0.033 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.605 │    0.033 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.319 │    0.010 │   0.000 │
├─────────┼─────────┼──────────

[ Info: RMSE Table
[ Info: MAE Table


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.605 │    0.033 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.605 │    0.033 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.319 │    0.010 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.319 │    0.010 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.706 │    0.042 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.706 │    0.042 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼

[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.601 │    0.033 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.601 │    0.033 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.315 │    0.010 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.315 │    0.010 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.605 │    0.033 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.605 │    0.033 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼

...

[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table



This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.

┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   9.647 │    0.334 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   9.647 │    0.334 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   6.814 │    0.230 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   6.814 │    0.230 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘


[ Info: RMSE Table
[ Info: MAE Table


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   9.577 │    0.332 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   9.577 │    0.332 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   6.726 │    0.229 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   6.726 │    0.229 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   1.847 │    0.053 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   1.847 │    0.053 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼

[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table


In [98]:
entries = length(keys(errs["val"]))
en_err = zeros(entries)
f_err = zeros(entries)
en_err_pure = zeros(entries)
f_err_pure = zeros(entries)

idx_order = [7, 8, 9, 4, 5, 6, 1, 2, 3]
sorted_keys = sort(collect(keys(errs["val"])))[idx_order]
for (i,tsetbset) in enumerate(sorted_keys)
    println(tsetbset) # uncomment to understand the structure 
    en_err[i] = errs["val"][tsetbset]["mae"]["default"]["E"]
    f_err[i] = errs["val"][tsetbset]["mae"]["default"]["F"]
    en_err_pure[i] = errs_pure["val"][tsetbset]["mae"]["default"]["E"]
    f_err_pure[i] = errs_pure["val"][tsetbset]["mae"]["default"]["F"]
end



en_err = reshape(en_err, (3, 3))' # transposed with (')
f_err = reshape(f_err, (3, 3))'
en_err_pure = reshape(en_err_pure, (3, 3))'
f_err_pure = reshape(f_err_pure, (3, 3))'

Si23_24.20
Si23_24.20.16
Si23_24.20.16.12
Si234_24.20
Si234_24.20.16
Si234_24.20.16.12
Si2345_24.20
Si2345_24.20.16
Si2345_24.20.16.12


3×3 adjoint(::Matrix{Float64}) with eltype Float64:
 0.0121678  0.0121678  0.0121678
 0.059883   0.0121673  0.0121673
 0.135367   0.0280062  0.0165793

In [99]:
open("./acejulia/si_errors/ind_vset$(vset)_en_err.dat", "w") do io
    for row in eachrow(en_err)
        for val in row
            @printf(io, "%.16f ", val)
        end
        println(io)
    end
end
open("./acejulia/si_errors/ind_vset$(vset)_f_err.dat", "w") do io
    for row in eachrow(f_err)
        for val in row
            @printf(io, "%.16f ", val)
        end
        println(io)
    end
end
open("./acejulia/si_errors/ind_vset$(vset)_en_err_pure.dat", "w") do io
    for row in eachrow(en_err_pure)
        for val in row
            @printf(io, "%.16f ", val)
        end
        println(io)
    end
end
open("./acejulia/si_errors/ind_vset$(vset)_f_err_pure.dat", "w") do io
    for row in eachrow(f_err_pure)
        for val in row
            @printf(io, "%.16f ", val)
        end
        println(io)
    end
end

## Plotting

# `Si345`

In [ ]:
vset = "Si2"
dsets = ["Si345"]
ecost = "50.0"
elossweight = parse(Float64, ecost)
weights = Dict("default" => Dict("E" => elossweight, "F" => 1.0))
basisvals = ["24.20","24.20.16","24.20.16.12"]
Vref = OneBody(:Si => -7881.32677981122) # Si2 isolated energy
datakeys = (energy_key = "energy", force_key = "forces")
val_data_dir = "./datasets/$(vset).xyz"; val_data = read_extxyz(val_data_dir)
val_atoms = [ACEpotentials.AtomsData(v; weights=weights, v_ref=Vref, datakeys...) for v in val_data]

pots = Dict()
pots_pure = Dict()
errs = Dict("train" => Dict(), "val" => Dict())
errs_pure = Dict("train" => Dict(), "val" => Dict())

for dset in dsets
    train_data_dir = "./datasets/$(dset).xyz"; train_data = read_extxyz(train_data_dir)
    train_atoms = [ACEpotentials.AtomsData(t; weights=weights, v_ref=Vref, datakeys...) for t in train_data]
    for basisval in basisvals
        pots["$(dset)_$(basisval)"] = load_potential("./acejulia/$(dset)/$(basisval)/ecost$(ecost)/potential.json")
        pots_pure["$(dset)_$(basisval)"] = load_potential("./acejulia/$(dset)_purify/$(basisval)/ecost$(ecost)/potential.json")

        # Errors
        errs["train"]["$(dset)_$(basisval)"] = ACEpotentials.linear_errors(train_atoms, pots["$(dset)_$(basisval)"])
        errs_pure["train"]["$(dset)_$(basisval)"] = ACEpotentials.linear_errors(train_atoms, pots_pure["$(dset)_$(basisval)"])
        errs["val"]["$(dset)_$(basisval)"] = ACEpotentials.linear_errors(val_atoms, pots["$(dset)_$(basisval)"])
        errs_pure["val"]["$(dset)_$(basisval)"] = ACEpotentials.linear_errors(val_atoms, pots_pure["$(dset)_$(basisval)"])

        # Dimers
        DSI = ACEpotentials.dimers(pots["$(dset)_$(basisval)"], [:Si,]; rr = range(0.529177, 7.0, length=200))
        D = ACEpotentials.dimers(pots_pure["$(dset)_$(basisval)"], [:Si,]; rr = range(0.529177, 7.0, length=200))
        export_dimers_to_dat(DSI, filename="./acejulia/$(dset)/$(basisval)/ecost$(ecost)/dimercurve.dat")
        export_dimers_to_dat(D, filename="./acejulia/$(dset)_purify/$(basisval)/ecost$(ecost)/dimercurve.dat")
    end
end


This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.


This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.



[ Info: RMSE Table


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  80.472 │    0.681 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  80.472 │    0.681 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  63.121 │    0.485 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  63.121 │    0.485 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘


[ Info: MAE Table


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  80.472 │    0.681 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  80.472 │    0.681 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  63.121 │    0.485 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  63.121 │    0.485 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │ 892.647 │    7.261 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │ 892.647 │    7.261 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼

...

[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table



This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.


This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.

┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  10.501 │    0.343 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  10.501 │    0.343 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   7.699 │    0.243 │   0.000 │
├─────────┼─────────┼──────────

[ Info: RMSE Table
[ Info: MAE Table


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  10.489 │    0.342 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  10.489 │    0.342 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   7.661 │    0.242 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   7.661 │    0.242 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │ 494.948 │    4.260 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │ 494.948 │    4.260 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼

[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table
